# Preprocess ICRP-107 to create dataset for radioactivedecay
### Introduction
This notebook creates a decay dataset for `radioactivedecay` from the data in <a href="http://www.icrp.org/publication.asp?id=ICRP%20Publication%20107">ICRP 107: Nuclear Decay Data for Dosimetric Calculations</a>.

The data is saved into the following files:
- `decay_data.npz`: NumPy arrays containing the strings of the radionuclides in the decay dataset, the half-lives and their time units, the progeny, branching fractions and decay modes of each radionuclide, and the days to year conversion number (NPZ NumPy compressed array format);
- `c_scipy.npz`: pre-calculated SciPy CSR sparse matrix *C* (Amaku et al. (2010)) (NPZ NumPy compressed array format);
- `c_inv_scipy.npz`: pre-calculated SciPy CSR sparse matrix *C<sup>-1</sup>* (inverse of *C*) (NPZ NumPy compressed array format);
- `c_sympy.pickle`: pre-calculated SymPy sparse matrix *C* for arbitrary-precision calculations (Python pickle format);
- `c_inv_sympy.pickle`: pre-calculated SymPy sparse matrix *C<sup>-1</sup>* for arbitrary-precision calculations (inverse of *C*) (Python pickle format);
- `decay_consts_sympy.pickle`: SymPy matrix or radionuclide decay constants (s<sup>-1</sup>) for arbitrary-precision calculations (inverse of *C*) (Python pickle format).
- `year_conversion_sympy.pickle`: SymPy representation of the days to year conversion number (Python pickle format).

### Initial set up and read in ICRP-107 data into a DataFrame
First load the necessary Python modules.

In [1]:
import io, pickle, re, shutil, zipfile
import fortranformat as ff
import numpy as np
import pandas as pd
import requests
from scipy import sparse
from sympy import Integer, S, N, Float, log, Matrix
from sympy.matrices import SparseMatrix 

Now we need to download and read in the data from the ICRP-07.NDX data file provided as a supplement to ICRP-107. First read a prepared CSV file listing all elements, their symbols and atomic numbers.

In [2]:
elements = pd.read_csv("element_list.csv", index_col="Symbol")[["Element","Z"]]
elements.head()

,Element,Z
Symbol,,
H,Hydrogen,1
He,Helium,2
Li,Lithium,3
Be,Beryllium,4
B,Boron,5


Define functions to
* convert half-life in units of μs, ms, m, h, d, y into seconds;
* create a human-readable string of the half-life;
* return atomic number and mass number from a radionuclide string.

In [3]:
year = 365.2422 # number of days in year used for conversion in ICRP-107 
                # (see JAERI 1347 & JAEA-Data/Code 2007-021)
num_nuclides = 1252

def convert_half_life(halflife, unit):
    units = {"μs":1.0E-6, "ms":1.0E-3, "s":1.0, "m":60.0, "h":60.0*60.0,
             "d":60.0*60.0*24.0, "y":60.0*60.0*24.0*year}
    return float(halflife)*units[unit]

def readable_half_life(halflife, unit):
    # very hacky code to format some of the half lives in ICRP-107 into more readable time units
    if unit == "s":
        if halflife[-3:] == "E-7":
            halflife = "0." + halflife[:-3].replace(".", "")
            unit = "μs"
        elif halflife[-3:] == "E-6":
            halflife = halflife[:-3]
            unit = "μs"
        elif halflife[-3:] == "E-5":
            if len(halflife[:-3]) == 3:
                halflife = halflife[:-3].replace(".", "")
            else:
                halflife = halflife[:-3].replace(".", "")[:2] + '.' + halflife[:-3].replace(".", "")[2:]
            unit = "μs"
        elif halflife[-3:] == "E-4":
            halflife = "0." + halflife[:-3].replace(".", "")
            unit = "ms"
        elif halflife[-3:] == "E-3":
            halflife = halflife[:-3]
            unit = "ms"
        elif halflife[-3:] == "E-2":
            if len(halflife[:-3]) == 3:
                halflife = halflife[:-3].replace(".", "")
            else:
                halflife = halflife[:-3].replace(".", "")[:2] + '.' + halflife[:-3].replace(".", "")[2:]
            unit = "ms"

    elif unit == "y":
        if halflife[-3:-1] == "E1":
            halflife = halflife[:-2] + "+" + halflife[-2:]
        
        if halflife[-3:] == "E+3":
            halflife = halflife[:-3]
            unit = "ky"
        elif halflife[-3:] == "E+4":
            if len(halflife[:-3]) == 3:
                halflife = halflife[:-3].replace(".", "")
            else:
                halflife = halflife[:-3].replace(".", "")[:2] + '.' + halflife[:-3].replace(".", "")[2:]
            unit = "ky"
        elif halflife[-3:] == "E+5":
            halflife = "0." + halflife[:-3].replace(".", "")
            unit = "My"
        elif halflife[-3:] == "E+6":
            halflife = halflife[:-3]
            unit = "My"
        elif halflife[-3:] == "E+7":
            if len(halflife[:-3]) == 3:
                halflife = halflife[:-3].replace(".", "")
            else:
                halflife = halflife[:-3].replace(".", "")[:2] + '.' + halflife[:-3].replace(".", "")[2:]
            unit = "My"
        elif halflife[-3:] == "E+8":
            halflife = "0." + halflife[:-3].replace(".", "")
            unit = "By"
        elif halflife[-3:] == "E+9":
            halflife = halflife[:-3]
            unit = "By"
        elif halflife[-4:] == "E+10":
            if len(halflife[:-4]) == 3:
                halflife = halflife[:-4].replace(".", "")
            else:
                halflife = halflife[:-4].replace(".", "")[:2] + '.' + halflife[:-4].replace(".", "")[2:]
            unit = "By"
        elif halflife[-4:] == "E+11":
            halflife = "0." + halflife[:-4].replace(".", "")
            unit = "Ty"
        elif halflife[-4:] == "E+12":
            halflife = halflife[:-4]
            unit = "Ty"
        elif halflife[-4:] == "E+13":
            if len(halflife[:-4]) == 3:
                halflife = halflife[:-4].replace(".", "")
            else:
                halflife = halflife[:-4].replace(".", "")[:2] + '.' + halflife[:-4].replace(".", "")[2:]
            unit = "Ty"
        elif halflife[-4:] == "E+14":
            halflife = "0." + halflife[:-4].replace(".", "")
            unit = "Py"
        elif halflife[-4:] == "E+15":
            halflife = halflife[:-4]
            unit = "Py"
        elif halflife[-4:] == "E+16":
            if len(halflife[:-4]) == 3:
                halflife = halflife[:-4].replace(".", "")
            else:
                halflife = halflife[:-4].replace(".", "")[:2] + '.' + halflife[:-4].replace(".", "")[2:]
            unit = "Py"
        elif halflife[-4:] == "E+17":
            halflife = halflife[:-4].replace(".", "")
            unit = "Py"
        
    return halflife + " " + unit

def get_Z_A(radionuclide):
    [Z, A] = radionuclide.split("-")
    Z = elements.loc[Z, "Z"]
    if A[-1].isalpha():
        A = A[:-1]
    return Z, int(A)

Prepare a pandas DataFrame for the ICRP-107 decay data.

In [4]:
icrp_col_names = ["Radionuclide", "Element", "Z", "A", "Metastable_state",
                  "Half_life", "Half_life_units", "Half_life_s",
                  "Half_life_readable","Num_decay_modes",
                  "Mode_1", "Fraction_1", "Progeny_1",
                  "Mode_2", "Fraction_2", "Progeny_2",
                  "Mode_3", "Fraction_3", "Progeny_3",
                  "Mode_4", "Fraction_4", "Progeny_4"]
icrp = pd.DataFrame(columns=icrp_col_names)

Download ICRP-107 Supplemental Material and read data from ICRP-07.NDX file line by line into the DataFrame.

In [5]:
url = "https://journals.sagepub.com/doi/suppl/10.1177/ANIB_38_3/suppl_file/P107JAICRP_38_3_Nuclear_Decay_Data_suppl_data.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

file_NDX = open("P 107 JAICRP 38(3) Nuclear Decay Data for Dosimetric Calculations(supplementary data)/ICRP-07.NDX", "r")
file_NDX.readline()

# fortran format of data in the ICRP-107 Index File (Table 1 footnote, ICRP107)
ffline = ff.FortranRecordReader("(a7,a8,a2,a8,3i7,i6,1x,3(a7,i6,e11.0,1x),a7,i6,e11.0,f7.0,2f8.0,3i4,i5,i4,e11.0,e10.0,e9.0)")
rows = []
for i in range(0, num_nuclides):
    line = ffline.read(file_NDX.readline())
    line = [i.strip() if isinstance(i,str) else i for i in line]
    add = {"Radionuclide": line[0]}
    add["Element"] = add["Radionuclide"].split("-")[0]
    add["Z"], add["A"] = get_Z_A(add["Radionuclide"])
    if add["Radionuclide"][-1].isalpha():
        add["Metastable_state"] = add["Radionuclide"][-1]
    add["Half_life"] = line[1]
    unit = line[2]
    if unit == "us":
        unit = "μs"
    add["Half_life_units"] = unit
    add["Half_life_s"] = convert_half_life(line[1], unit)
    add["Half_life_readable"] = readable_half_life(line[1], unit)

    # parse decay modes and progeny
    modes = re.findall(r"(A|B\-|ECB\+|EC|IT|SF)",line[3])
    j=0
    while j < 4 and line[8+j*3] != "":
        add["Fraction_" + str(j+1)] = str(line[8+2+j*3])
        add["Progeny_" + str(j+1)] = line[8+j*3]
        if add["Progeny_" + str(j+1)] == "SF":
            add["Mode_" + str(j+1)] = "SF"
        else:
            Z, A = get_Z_A(add["Progeny_" + str(j+1)])
            if add["Z"] == Z and add["A"] == A: add["Mode_" + str(j+1)] = "IT"
            elif add["Z"] - 2 == Z and add["A"] - 4 == A: add["Mode_" + str(j+1)] = "α"
            elif add["Z"] + 1 == Z and add["A"] == A: add["Mode_" + str(j+1)] = "β-"
            elif add["Z"] - 1 == Z and add["A"] == A:
                if "EC" in modes: add["Mode_" + str(j+1)] = "EC"
                else: add["Mode_" + str(j+1)] = "β+ & EC"
        j += 1
    add["Num_decay_modes"] = j

    rows.append(add)
icrp = icrp.append(rows, ignore_index=True)
file_NDX.close()

shutil.rmtree("P 107 JAICRP 38(3) Nuclear Decay Data for Dosimetric Calculations(supplementary data)")

Remove NaN values, set DataFrame index to the radionuclide string, and check completed DataFrame. Export completed DataFrame to CSV file for analysis elsewhere.

In [6]:
icrp = icrp.replace(np.nan, "", regex=True)
icrp.set_index("Radionuclide", inplace=True)
icrp.to_csv("icrp.csv", index=True)
icrp.head()

,Element,Z,A,Metastable_state,Half_life,Half_life_units,Half_life_s,Half_life_readable,Num_decay_modes,Mode_1,...,Progeny_1,Mode_2,Fraction_2,Progeny_2,Mode_3,Fraction_3,Progeny_3,Mode_4,Fraction_4,Progeny_4
Radionuclide,,,,,,,,,,,,,,,,,,,,,
Ac-223,Ac,89,223,,2.10,m,1.260000e+02,2.10 m,1,α,...,Fr-219,,,,,,,,,
Ac-224,Ac,89,224,,2.78,h,1.000800e+04,2.78 h,2,EC,...,Ra-224,α,0.091,Fr-220,,,,,,
Ac-225,Ac,89,225,,10.0,d,8.640000e+05,10.0 d,1,α,...,Fr-221,,,,,,,,,
Ac-226,Ac,89,226,,29.37,h,1.057320e+05,29.37 h,3,β-,...,Th-226,EC,0.17,Ra-226,α,6e-05,Fr-222,,,
Ac-227,Ac,89,227,,21.772,y,6.870574e+08,21.772 y,2,β-,...,Th-227,α,0.0138,Fr-223,,,,,,


### Order ICRP DataFrame so progeny always come below their parent
The radionuclides need to be ordered so that the progeny (daughters) are always lower in the DataFrame than their parent. This is so the subsequent matrices that we create are lower triangular.

To achieve this we first count how many times each radioactive decay process occurs in the ICRP-107 dataset.

In [7]:
print("β+ or electron capture:", icrp.stack().value_counts()["β+ & EC"]
      + icrp.stack().value_counts()["EC"])
print("β-:", icrp.stack().value_counts()["β-"])
print("α:", icrp.stack().value_counts()["α"])
print("Isomeric Transition (IT):", icrp.stack().value_counts()["IT"])
print("Spontaneous Fission (SF):", (icrp.stack().value_counts()["SF"]/2).astype(np.int64))

β+ or electron capture: 684
β-: 539
α: 183
Isomeric Transition (IT): 178
Spontaneous Fission (SF): 28


The outcomes of these decay processes are as follows:
- β+ or electron capture (EC): $\mathrm{^{A}_{Z}X} \rightarrow \mathrm{^{A}_{Z-1}Y}$
- β- decay: $\mathrm{^{A}_{Z}X} \rightarrow \mathrm{^{A}_{Z+1}Y}$
- α decay: $\mathrm{^{A}_{Z}X} \rightarrow \mathrm{^{A-4}_{Z-2}Y}$
- IT decay: $\mathrm{^{Am}_{Z}X} \rightarrow \mathrm{^{A}_{Z}X}$ or $\mathrm{^{An}_{Z}X} \rightarrow \mathrm{^{A}_{Z}X}$
- SF decay: The ICRP-107 dataset does not contain data for the outcomes (progeny) from spontaneous fission decays

We order by decreasing mass number (A), followed by decreasing atomic number (Z) (as there are more Beta+ and EC decays than Beta- decays), then by decreasing isomer index (n, m, ground state).

In [8]:
icrp.sort_values(by=["A", "Z", "Metastable_state"], inplace=True, ascending=[False, False, False])
icrp.head()

,Element,Z,A,Metastable_state,Half_life,Half_life_units,Half_life_s,Half_life_readable,Num_decay_modes,Mode_1,...,Progeny_1,Mode_2,Fraction_2,Progeny_2,Mode_3,Fraction_3,Progeny_3,Mode_4,Fraction_4,Progeny_4
Radionuclide,,,,,,,,,,,,,,,,,,,,,
Fm-257,Fm,100,257,,100.5,d,8683200.0,100.5 d,2,α,...,Cf-253,SF,0.0021,SF,,,,,,
Fm-256,Fm,100,256,,157.6,m,9456.0,157.6 m,2,α,...,Cf-252,SF,0.919,SF,,,,,,
Es-256,Es,99,256,,25.4,m,1524.0,25.4 m,1,β-,...,Fm-256,,,,,,,,,
Fm-255,Fm,100,255,,20.07,h,72252.0,20.07 h,2,α,...,Cf-251,SF,2.3e-07,SF,,,,,,
Es-255,Es,99,255,,39.8,d,3438720.0,39.8 d,3,β-,...,Fm-255,α,0.08,Bk-251,SF,4.5e-05,SF,,,


Now it is necessary to correct the positions of the remaining radionuclides that are still incorrectly ordered. This is achieved by looping over all the radionuclides in the table, and checking if their progeny are located lower in the table or not. If not, the parent and progeny row positions are switched. This takes a few passes until all progeny are correctly located below their parents.

In [9]:
nuclide_list = list(icrp.index)
swapping = 1
while swapping >= 1:
    swaps = 0
    for parent in nuclide_list:
        for i in range(0, icrp.at[parent, "Num_decay_modes"]):
            if (icrp.at[parent, "Mode_" + str(i+1)] in ["stable", "SF"]): continue
            progeny = icrp.at[parent, "Progeny_" + str(i+1)]
            if (progeny not in nuclide_list): continue
            j = nuclide_list.index(parent)
            k = nuclide_list.index(progeny)
            if  j > k:
                nuclide_list[j], nuclide_list[k] = nuclide_list[k], nuclide_list[j]
                icrp = icrp.reindex(index=nuclide_list)
                swaps +=1
    print("Iteration", swapping, "number of swaps:", swaps)
    swapping += 1
    if swaps == 0: swapping = 0

Iteration 1 number of swaps: 265
Iteration 2 number of swaps: 81
Iteration 3 number of swaps: 22
Iteration 4 number of swaps: 4
Iteration 5 number of swaps: 0


The sorted DataFrame looks like this. Note this is just one of many possible solutions for sorting the DataFrame.

In [10]:
icrp.head()

,Element,Z,A,Metastable_state,Half_life,Half_life_units,Half_life_s,Half_life_readable,Num_decay_modes,Mode_1,...,Progeny_1,Mode_2,Fraction_2,Progeny_2,Mode_3,Fraction_3,Progeny_3,Mode_4,Fraction_4,Progeny_4
Radionuclide,,,,,,,,,,,,,,,,,,,,,
Fm-257,Fm,100,257,,100.5,d,8683200.0,100.5 d,2,α,...,Cf-253,SF,0.0021,SF,,,,,,
Es-256,Es,99,256,,25.4,m,1524.0,25.4 m,1,β-,...,Fm-256,,,,,,,,,
Fm-256,Fm,100,256,,157.6,m,9456.0,157.6 m,2,α,...,Cf-252,SF,0.919,SF,,,,,,
Cf-255,Cf,98,255,,85,m,5100.0,85 m,1,β-,...,Es-255,,,,,,,,,
Es-255,Es,99,255,,39.8,d,3438720.0,39.8 d,3,β-,...,Fm-255,α,0.08,Bk-251,SF,4.5e-05,SF,,,


### Make the *&Lambda;* matrix
Now we make the sparse lower triangular matrix *&Lambda;*, which captures the decay pathways and branching relations between the radionuclides. _&Lambda;_ is set up based on Eq. (6) in Amaku et al. (2010). The diagonal elements are all *-&lambda;<sub>jj</sub>*, i.e. negative decay constant for each radionuclide. The off-diagonal elements are all of the form *BF<sub>ij</sub>&times;&lambda;<sub>jj</sub>* for *i* > *j*, where *BF<sub>ij</sub>* is the branching fraction from parent *j* to progeny *i*. The non-zero elements beneath the *jj*<sup>th</sup> element in each column are first progeny of radionuclide *j*.

In [11]:
rows = np.array([], dtype=np.int)
cols = np.array([], dtype=np.int)
data = np.array([], dtype=np.float)
ln2 = np.log(2)

for parent in nuclide_list:
    j = nuclide_list.index(parent)
    rows = np.append(rows, [j])
    cols = np.append(cols, [j])
    lambd = ln2/icrp.at[parent, "Half_life_s"]
    data = np.append(data, -lambd)
    for d in range(0, icrp.at[parent, "Num_decay_modes"]):
        if (icrp.at[parent, "Mode_" + str(d+1)] in ["stable", "SF"]): continue
        progeny = icrp.at[parent, "Progeny_" + str(d+1)]
        if (progeny not in nuclide_list): continue
        i = nuclide_list.index(progeny)
        rows = np.append(rows, [i])
        cols = np.append(cols, [j])
        data = np.append(data, [lambd*float(icrp.at[parent, "Fraction_" + str(d+1)])])

lambda_mat = sparse.csc_matrix((data, (rows, cols)))

### Calculate the matrices *C* and *C<sup>-1</sup>*
We now need to make the sparse matrices *C* and *C<sup>-1</sup>*, which are given by Eqs. (10) and (13) in Amaku et al. (2010), respectively. The diagonal elements of both matrices are 1. *C* and *C<sup>-1</sup>* differ from *&Lambda;* in that there are non-zero elements beneath the *jj*<sup>th</sup> element in each column for all progeny of *j*, i.e. everything in its full decay chain, not just the immediate daughters.

Therefore we have to find all the progeny in the decay chain of each radionuclide. We do this by looping backwards over each column in *&Lambda;* to build up lists of the radionuclides in the decay chain of each parent. We then set up the basic structure (i.e. define the non-zero elements) of sparse matrices *C* and *C<sup>-1</sup>*.

In [12]:
rows_dict = {}
for i in range(num_nuclides-1, -1, -1):
    a,_ = lambda_mat[:,i].nonzero()
    b = a
    for j in a:
        if j > i:
            b = np.unique(np.concatenate((b,rows_dict[j])))
    rows_dict[i] = b

rows_c = np.array([], dtype=np.int)
cols_c = np.array([], dtype=np.int)
for i in range(0, num_nuclides):
    rows_c = np.concatenate((rows_c,rows_dict[i]))
    cols_c = np.concatenate((cols_c,np.array([i]*len(rows_dict[i]))))

c = sparse.csc_matrix((np.array([0.0]*rows_c.size, dtype=np.float64), (rows_c, cols_c)))
c_inv = sparse.csc_matrix((np.array([0.0]*rows_c.size, dtype=np.float64), (rows_c, cols_c)))

Now calculate *C* and *C<sup>-1<sup>*. Note that only the non-zero elements of *C<sub>kj</sub>* and *C<sup>-1</sup><sub>kj</sub>*  need to be considered for the sums in Eqs. (10) and (13) of Amaku et al. (2010).
    
Calculate *C*, and highlight any cases where the relative difference of the decay constants of two radionuclides in the same decay chain is less than 0.001 (as this could lead to numerical precision issues):

In [13]:
for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: c[i,i] = 1.0
    else:
        sigma = 0.0
        for k in rows_dict[j]:
            if k == i: break
            sigma += lambda_mat[i,k]*c[k,j]
        c[i,j] = sigma/(lambda_mat[j,j]-lambda_mat[i,i])

        if abs((lambda_mat[j,j]-lambda_mat[i,i])/lambda_mat[j,j]) < 1E-3: print(nuclide_list[i], nuclide_list[j])

There are no cases where radionuclides in the same decay chain have decay constants that are too similar (for the radionuclides in ICRP-107). Now proceed to calculate *C<sup>-1</sup>*:

In [14]:
for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: c_inv[i,i] = 1.0
    else:
        sigma = 0.0
        for k in rows_dict[j]:
            if k == i: break
            sigma -= c[i,k]*c_inv[k,j]
        c_inv[i,j] = sigma

### Calculate SymPy versions of the matrices for arbitrary-precision calculations
We now calculate SymPy versions of *C* and *C<sup>-1<sup>* for arbitrary-precision calculations. First define some functions for processing the data into SymPy objects:

In [15]:
year_sympy = S(3652422)/10000

def to_rational(number):
    """
    Converts string representation of a number to a SymPy object.
    """

    if 'e' in number or 'E' in number:
        if 'e' in number:
            end = number.split('e')[1]
            number = number.split('e')[0]
        else:
            end = number.split('E')[1]
            number = number.split('E')[0]
        parts = number.split('.')
        if len(parts) == 1: parts.append('')
        if end[0] == '+':
            multiply = 1
            factor = S(10**int(end.lstrip('+')))
        else:
            multiply = 0
            factor = S(10**int(end.lstrip('-')))
        denom = S(10**len(parts[1]))
        parts[0] = parts[0].lstrip('0')
        if len(parts[0]) == 0: parts[1] = parts[1].lstrip('0')
        if multiply == 1:
            return S(parts[0]+parts[1])*factor/denom
        else: return S(parts[0]+parts[1])/(denom*factor)
    parts = number.split('.')
    if len(parts) == 1: parts.append('')
    denom = S(10**len(parts[1]))
    parts[0] = parts[0].lstrip('0')
    if len(parts[0]) == 0: parts[1] = parts[1].lstrip('0')
    return S(parts[0]+parts[1])/denom

def convert_half_life_sympy(halflife, unit):
    """
    Conversion of SymPy half-life into seconds.
    """

    units = {'μs':S(1)/1000000,
             'ms':S(1)/1000,
             's':S(1),
             'm':S(60),
             'h':S(3600),
             'd':S(86400),
             'y':S(86400)*year_sympy
            }
    return halflife*units[unit]

Now make a SymPy version of the *&Lambda;* matrix:

In [16]:
lambda_mat_sympy = SparseMatrix.zeros(num_nuclides, num_nuclides)
lambdas_sympy = Matrix.zeros(num_nuclides, 1)

for parent in nuclide_list:
    j = nuclide_list.index(parent)
    hl_sympy = convert_half_life_sympy(to_rational(icrp.at[parent, 'Half_life']),
                                       icrp.at[parent, 'Half_life_units'])
    lambd = log(2)/hl_sympy
    lambda_mat_sympy[j, j] = -lambd
    lambdas_sympy[j] = lambd
    for d in range(0, icrp.at[parent, 'Num_decay_modes']):
        if (icrp.at[parent, 'Mode_'+str(d+1)] in ['stable', 'SF']): continue
        progeny = icrp.at[parent, 'Progeny_'+str(d+1)]
        if (progeny not in nuclide_list): continue
        i = nuclide_list.index(progeny)
        lambda_mat_sympy[i, j] = lambd*to_rational(icrp.at[parent, 'Fraction_'+str(d+1)])

Now make a SymPy version of the *C* and *C<sup>-1</sup>* matrix:

In [17]:
c_sympy = SparseMatrix.zeros(num_nuclides, num_nuclides)
c_inv_sympy = SparseMatrix.zeros(num_nuclides, num_nuclides)

for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: c_sympy[i,i] = Integer(1)
    else:
        sigma = Integer(0)
        for k in rows_dict[j]:
            if k == i: break
            sigma += lambda_mat_sympy[i,k]*c_sympy[k,j]
        c_sympy[i,j] = sigma/(lambda_mat_sympy[j,j]-lambda_mat_sympy[i,i])

for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: c_inv_sympy[i,i] = Integer(1)
    else:
        sigma = Integer(0)
        for k in rows_dict[j]:
            if k == i: break
            sigma -= c_sympy[i,k]*c_inv_sympy[k,j]
        c_inv_sympy[i,j] = sigma 

### Save the outputs
Write out file containing NumPy arrays with the radionuclide names, the half-lives, their time units, human-readable half-life strings, the progeny, branching fractions and decay modes of each radionuclide, and the days to year conversion number. Write output files containing *C* and *C<sup>-1</sup>* in the SciPy and SymPy sparse formats. Write out the SymPy versions of the decay constants.

In [18]:
hldata = np.array([(np.float64(a),b,c) for a, b, c in zip(icrp["Half_life"], icrp["Half_life_units"], icrp["Half_life_readable"])], dtype=object)

prog_bfs_modes = np.array([{}]*len(nuclide_list))
for i in range(0, len(nuclide_list)):
    bfs = {}
    modes = {}
    for d in range(0, icrp.at[nuclide_list[i], "Num_decay_modes"]):
        progeny = icrp.at[nuclide_list[i], "Progeny_" + str(d+1)]
        bfs[progeny] = float(icrp.at[nuclide_list[i], "Fraction_" + str(d+1)])
        modes[progeny] = icrp.at[nuclide_list[i], "Mode_" + str(d+1)]
    bfs = {key: value for key, value in sorted(bfs.items(), key=lambda x: x[1], reverse=True)}
    
    prog_bfs_modes[i] = {progeny: [bf, modes[progeny]] for progeny, bf in bfs.items()}

np.savez_compressed("./decay_data.npz", radionuclides=np.array(nuclide_list),
                    hldata=hldata, prog_bfs_modes=prog_bfs_modes,
                    year_conv=year)

# Write out SciPy sparse matrices (convert to CSR format)
sparse.save_npz("./c_scipy.npz", c.tocsr())
sparse.save_npz("./c_inv_scipy.npz", c_inv.tocsr())

# Write out SymPy objects to pickle files
with open("c_sympy.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(c_sympy))
with open("c_inv_sympy.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(c_inv_sympy))
with open("decay_consts_sympy.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(lambdas_sympy))
with open("year_conversion_sympy.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(year_sympy))